In [ ]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.row_number

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.appName("Scala Analytic Functions").getOrCreate()
 
import spark.implicits._

val text = spark.sparkContext.textFile("/home/jovyan/Resources/tx.csv")

case class Transaction(date: String, desc: String, amount: Int)

val txns = text.map(s => s.split(",")).map(a => Transaction(a(0), a(1), (a(2).toFloat).round))
  .filter(t => t.amount < 0).toDF

// Use Spark Window functions

val w = Window.partitionBy($"date").orderBy($"amount")
val rnkd = txns.withColumn("rank", row_number.over(w))
val seld = rnkd.filter($"rank" < 4).drop("rank")

seld.show
